In [1]:
import torch
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
import wandb

In [2]:
from dotenv import dotenv_values
import os

envs = ["secret.env"]

for fenv in envs:
    file = os.path.join("env", fenv)
    config = dotenv_values(file)  # load sensitive variables
    print(config.keys())
    for c, v in config.items():
        os.environ[c] = v

odict_keys(['WANDB_API_KEY', 'WANDB_PROJECT'])


In [3]:
import wandb
import os

wandb_key = os.environ["WANDB_API_KEY"]
wandb.login(key=wandb_key)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: wilber-quito (deepsat). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import json


class WANDBConfig:
    def __enter__(self):
        config = {}
        with open("config/autoencoder-ssl.config.json") as f:
            config = json.load(f)
            if config["accelerate"]:
                device = "cuda" if torch.cuda.is_available() else "cpu"
                config["device"] = device
            else:
                config["device"] = "cpu"

        wandb.init(job_type="autoencoder-fit", config=config)

    def __exit__(self, exc_type, exc_val, exc_tb):
        wandb.finish()

In [5]:
import torch


class ClearCache:
    def __enter__(self):
        torch.cuda.empty_cache()

    def __exit__(self, exc_type, exc_val, exc_tb):
        torch.cuda.empty_cache()

In [6]:
def dataloader():
    # Transforms images to a PyTorch Tensor
    tensor_transform = transforms.ToTensor()

    # Download the MNIST Dataset
    dataset = datasets.MNIST(
        root="./data", train=True, download=True, transform=tensor_transform
    )

    # DataLoader is used to load the dataset
    # for training
    loader = torch.utils.data.DataLoader(
        dataset=dataset, batch_size=wandb.config.batch_size, shuffle=True
    )

    return loader

In [7]:
import torch


class AE(torch.nn.Module):
    def __init__(self, latence_space):
        super().__init__()

        self.latence_space = latence_space

        # Building an linear encoder with Linear
        # layer followed by Relu activation function
        # 784 ==> 9
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(28 * 28, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 18),
            torch.nn.ReLU(),
            torch.nn.Linear(18, self.latence_space),
        )

        # Building an linear decoder with Linear
        # layer followed by Relu activation function
        # The Sigmoid activation function
        # outputs the value between 0 and 1
        # 9 ==> 784
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(self.latence_space, 18),
            torch.nn.ReLU(),
            torch.nn.Linear(18, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 28 * 28),
            torch.nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [8]:
def train(model, optimizer, loss_fn, loader):

    outputs = []
    losses = []

    model = model.to(wandb.config.device)

    for epoch in tqdm(range(wandb.config.epochs)):
        for image, _ in loader:

            # Reshaping the image to (-1, 784)
            image = image.reshape(-1, 28 * 28)
            image = image.to(wandb.config.device)

            # Output of Autoencoder
            reconstructed = model(image)

            # Calculating the loss function
            loss = loss_fn(reconstructed, image)

            # The gradients are set to zero,
            # the gradient is computed and stored.
            # .step() performs parameter update
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            wandb.log({"loss": loss, "epoch": epoch})

            # Storing the losses in a list for plotting
            losses.append(loss)
            outputs.append((epoch, image, reconstructed))

    return losses, outputs

In [9]:
with WANDBConfig() as cf, ClearCache() as cc:
    # Dataloader
    loader = dataloader()

    # Model Initialization
    model = AE(latence_space=wandb.config.latence_space)

    # Validation using MSE Loss function
    loss_function = torch.nn.MSELoss()

    # Using an Adam Optimizer with lr = 0.01
    optimizer = torch.optim.Adam(
        model.parameters(), lr=wandb.config.lr, weight_decay=1e-8
    )
    losses, outputs = train(model, optimizer, loss_function, loader)

100%|██████████| 20/20 [20:23<00:00, 61.19s/it]


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
loss,█▂▂▁▃▁▂▂▂▂▂▁▂▂▂▂▂▁▁▁▁▂▂▂▂▂▂▁▁▁▂▂▁▂▂▂▂▂▂▂
epoch,19
loss,0.06366


In [11]:
model.state_dict()

OrderedDict([('encoder.0.weight',
              tensor([[-4.9829e-37, -5.0001e-37,  4.9405e-37,  ...,  4.9165e-37,
                       -4.9241e-37,  4.9297e-37],
                      [-4.9588e-37,  4.9647e-37, -4.9861e-37,  ...,  4.9390e-37,
                        5.0241e-37, -4.9897e-37],
                      [ 5.0071e-37,  5.0190e-37,  4.9101e-37,  ...,  4.9373e-37,
                        5.0291e-37,  5.0374e-37],
                      ...,
                      [-4.9953e-37,  4.9690e-37,  5.0382e-37,  ..., -4.9224e-37,
                        4.9445e-37, -5.0217e-37],
                      [-4.9762e-37,  5.0191e-37,  4.9994e-37,  ...,  4.9795e-37,
                       -4.9189e-37, -4.9287e-37],
                      [ 4.9436e-37, -5.0123e-37,  5.0419e-37,  ...,  4.9288e-37,
                       -4.9143e-37,  5.0039e-37]], device='cuda:0')),
             ('encoder.0.bias',
              tensor([ 4.9200e-37, -5.0343e-37,  4.9152e-37, -4.9701e-37,  4.9527e-37,
              

In [14]:
torch.save(model.state_dict(), "autoencoder.pth")